# 역전파를 포함한 신경망 학습 및 추론 구현
- Softmax-with-Loss 계층
- 오차역전파법 구현하기

### Softmax-with-loss 계층
- Softmax와 Cross Entropy Error가 같이 조합된 경우, 역전파가 단순히 y - t의 값 즉, 오차를 전달한다

In [9]:
import numpy as np
def softmax(x):
    c = np.max(x)
    exp_x = np.exp(x-c)
    sum_exp_x = np.sum(exp_x)
    return exp_x / sum_exp_x

In [ ]:
def cross_entropy_error(y,t):
    if y.ndim == 1:
        t = t.reshape(1,t.size)
        y = y.reshape(1,y.size)
    batch_size = y.shape[0]
    return -np.sum(t*(np.log(y+1e-7))) / batch_size

In [8]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None # 정답 레이블(원-핫 벡터)
        
    def forward(self,x,t): # Softmax의 출력값 y와 기존 정답라벨 t 비교
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y,self.t)
        return loss
    def backward(self,dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size # 배치의 수(Batch size)로 나눠서 데이터 1개당 오차를 전달
        return dx

- Q. 배치의 수(Batch size)로 나눠서 데이터 1개당 오차를 전달하는 이유는 무엇일까?
- CEE를 계산할 때 데이터 개수(batch_size)로 나눠 정규화하여 데이터 1개당 오차를 계산해 합했기 때문

### 오차역전파를 적용한 신경망 구현하기

In [60]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad

In [61]:
class ReLU:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [62]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        
        self.x = None
        self.original_x_shape = None
        # 가중치와 편향 매개변수의 미분
        self.dW = None # 생성자에서 초기화해야 나중에 불러올 수 있음
        self.db = None

    def forward(self, x):
        # 텐서 대응
        self.original_x_shape = x.shape
        x = x.reshape(x.shape[0], -1)
        self.x = x

        out = np.dot(self.x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout) 
        self.db = np.sum(dout, axis=0)
        
        dx = dx.reshape(*self.original_x_shape)  # 입력 데이터 모양 변경(텐서 대응)
        return dx

In [63]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from collections import OrderedDict

In [64]:
class TwoLayerNet: # 해당 TwoLayerNet은 수치미분 방식이 아닌 해석학적 방법(역전파)으로 기울기를 구한다
    def __init__(self,input_size,hidden_size,output_size,
                weight_init_std = 0.01):
        # 가중치 초기화 - params 라는 딕셔너리에 보관되어 있다
        self.params = {}
        self.params["W1"] = weight_init_std * np.random.randn(input_size,hidden_size)
        self.params["b1"] = np.zeros(hidden_size)
        self.params["W2"] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.params["b2"] = np.zeros(output_size)
        
        # 신경망의 각 기능을 계층으로 구현할 것이다.
        # 행렬곱 계층 2개 / 활성화함수(ReLU) 계층 1개 / SoftmaxwithLoss 계층 1개
        
        self.layers = OrderedDict() # 순서가 있는 딕셔너리
        self.layers["Affine1"] = Affine(self.params["W1"],self.params["b1"]) # Affine 클래스의 인스턴스를 보관
        self.layers["ReLU"] = ReLU()
        self.layers["Affine2"] = Affine(self.params["W2"],self.params["b2"])
        
        self.lastlayer = SoftmaxWithLoss() # 마지막 계층
        
    def predict(self,x):
        for layer in self.layers.values(): # layers.values에는 Affine1 계층, ReLU계층, Affine2 계층이 순차적으로
            x = layer.forward(x)
            
        return x
    
    def loss(self,x,t):
        y = self.predict(x)
        
        return self.lastlayer.forward(y,t) # CEE 산출
    
    def accuracy(self,x,t):
        y = self.predict(x)
        y = np.argmax(y,axis=1) # 행 별로 최댓값의 인덱스 추출
        if t.ndim != 1: # 원-핫 인코디여부 판단
            t = np.argmax(t,axis=1)
        else:
            pass # 원-핫 인코딩이 되지 않은 경우는 t 자체가 정답 레이블 정보
        
        return np.sum(y == t) / float(x.shape[0])
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self,x,t): # 역전파 해석학적 미분
        # 순전파 예측하고 loss 계산까지 선행되어야 W 가 할당된다
        self.loss(x,t)
            
        # 역전파
        dout = 1
        dout = self.lastlayer.backward(dout) # self.loss(x,t)를 시행하면 self.predict(x) 하게 되니까 y,t가 할당됨
        
        layers = list(self.layers.values())
        layers.reverse() # 순전파 시행했던 것 순서 뒤집어서 역전파 시행
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과 저장 / 기울기 값 저장
        grads = {}
        grads["W1"] = self.layers["Affine1"].dW
        grads["b1"] = self.layers["Affine1"].db
        grads["W2"] = self.layers["Affine2"].dW
        grads["b2"] = self.layers["Affine2"].db
        
        return grads

- 신경망의 각 기능을 계층으로 구현할 것이다.
- 행렬곱 계층 2개 / 활성화함수(ReLU) 계층 1개 / SoftmaxwithLoss 계층 1개
- 행렬곱1 -> ReLU -> 행렬곱2 -> SoftmaxwithLoss  
- 역전파를 시행하기 위해서는 predict와 loss 계산이 순차적으로 되어 있어야 한다

  ### 오차역전파로 구한 기울기 검증하기
  - 해석학적 미분과 수치학적 미분의 결과 비교

In [65]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽어오기
(x_train,t_train),(x_test,t_test) = load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(input_size=784,hidden_size=50,output_size=10)

batch_size = 3
x_batch = x_train[:batch_size] # batch_size 3개
t_batch = t_train[:batch_size]

In [68]:
grd_backprop = network.gradient(x_batch,t_batch) # 역전파 방식으로 기울기 계산
grd_numerical = network.numerical_gradient(x_batch,t_batch) # 수치미분 방식으로 기울기 계산

# 기울기 차이의 절댓값의 평균

for key in grd_numerical.keys():
    diff = np.average(np.abs(grd_numerical[key] - grd_backprop[key]))
    print("매개변수",key,"의 차이 ", diff)

매개변수 W1 의 차이  4.302967606482729e-10
매개변수 b1 의 차이  2.6441861126786507e-09
매개변수 W2 의 차이  5.019841105248321e-09
매개변수 b2 의 차이  1.3966660540459807e-07


### 오차역전파를 적용한 신경망 구현하기
- 실제 학습 진행해보기

In [83]:
import sys,os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# 데이터 읽어온 뒤에 훈련, 평가데이터로 나누기
(x_train,t_train),(x_test,t_test) = load_mnist(normalize = True, one_hot_label = True)

# 신경망 구현
network = TwoLayerNet(input_size=784,hidden_size=50,output_size=10)
loss_lst = [] ; train_acc_lst = [] ; test_acc_lst = []

# 학습진행
iters_num = 10000 ; batch_size = 100 ; lrn_rate = 0.1 ; train_size = x_train.shape[0]
iter_per_epoch = max(train_size/batch_size,1) # 1epoch당 반복횟수

for i in range(iters_num): # 학습자체는 10,000회 = iters_num 만큼 진행
    mask = np.random.choice(train_size,batch_size)
    x_batch = x_train[mask]
    t_batch = t_train[mask]
    
    # 오차 역전파로 기울기 구하기
    grad = network.gradient(x_batch,t_batch)
    
    # 경사하강법으로 기울기 갱신
    for key in ["W1","W2","b1","b2"]:
        grad[key] -= lrn_rate * grad[key]
        
        loss = network.loss(x_batch,t_batch) # 손실함수 기록하기
        loss_lst.append(loss)
        
    if i%iter_per_epoch == 0: # 1epoch 마다 정확도 기록하기
        train_acc = network.accuracy(x_batch,t_batch)
        train_acc_lst.append(train_acc)
        
        test_acc = network.accuracy(x_test,t_test)
        test_acc_lst.append(test_acc)
        print(i,"번째 반복",train_acc,test_acc)

0 번째 반복 0.09 0.0756

600 번째 반복 0.09 0.0756

1200 번째 반복 0.07 0.0756

1800 번째 반복 0.06 0.0756

2400 번째 반복 0.12 0.0756

3000 번째 반복 0.05 0.0756

3600 번째 반복 0.06 0.0756

4200 번째 반복 0.06 0.0756

4800 번째 반복 0.08 0.0756

5400 번째 반복 0.08 0.0756

6000 번째 반복 0.08 0.0756

6600 번째 반복 0.06 0.0756

7200 번째 반복 0.13 0.0756

7800 번째 반복 0.05 0.0756

8400 번째 반복 0.06 0.0756

9000 번째 반복 0.07 0.0756

9600 번째 반복 0.06 0.0756

